## Ce code met en place la première partie d'un système RAG (Retrieval-Augmented Generation) : l'extraction de données et la création d'une base de données vectorielle.
## ⚠️ Avertissement de sécurité :  
 Vous avez laissé votre clé API (LLAMA_API_KEY) visible dans le code. Je vous conseille vivement de la révoquer et d'en générer une nouvelle, car n'importe qui ayant accès à ce texte peut l'utiliser.

 ### 1. Analyse détaillée par étape

Étape 1 : Importation des librairies

In [1]:
import os
from llama_parse import LlamaParse
from llama_parse.base import ResultType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from llama_cloud_services.parse.utils import Language
from langchain_community.embeddings.ollama import OllamaEmbeddings

### Explication :
 Ce bloc charge tous les outils nécessaires.
LlamaParse : Un outil puissant pour extraire du texte depuis des fichiers complexes (PDF), particulièrement bon pour lire les tableaux et les structures.
Chroma : La base de données vectorielle qui va stocker les informations.
OllamaEmbeddings : Le modèle qui va transformer le texte en vecteurs (listes de nombres) pour que la machine comprenne le sens des phrases.
### Interprétation :
 On prépare la "boîte à outils". On choisit ici d'utiliser Ollama (local) pour les embeddings et Chroma pour le stockage, ce qui permet une architecture privée et efficace.

Étape 2 : Configuration du Parser Llama

In [2]:
# Configuration de l'API Key
LLAMA_API_KEY = "llx-a2C7FgYfP1hzX3pXuvtdaNmexAqsuRnJIJ2G6MjbBrfuS3QY"  # Remplacez par votre clé
os.environ["LLAMA_CLOUD_API_KEY"] = LLAMA_API_KEY

# Initialisation du parser pour l'arabe"
parser_ar = LlamaParse(
    result_type=ResultType.MD,
    language=Language.ARABIC,
    verbose=True)
print("✅ Parser arabe initialisé avec succès")

✅ Parser arabe initialisé avec succès


### Explication : 
On configure la clé API pour le service LlamaCloud. On initialise ensuite le parser en spécifiant que le document est en Arabe (Language.ARABIC) et qu'on veut le résultat au format Markdown (ResultType.MD).
### Interprétation : 
C'est une étape critique pour votre document "Wa7ch.pdf". Les PDF en arabe sont souvent très mal lus par les extracteurs classiques. LlamaParse utilise de l'IA pour reconstruire correctement le texte arabe et la structure du document.

Étape 3 : Extraction et Conversion (PDF -> Markdown)

In [3]:

import nest_asyncio
nest_asyncio.apply()

pdf_files = [
    ("Wa7ch.pdf", parser_ar)
   
    
    # ajoute autant que nécessaire
]

# Nom du fichier Markdown de sortie
output_filename = "Wa7ch.md"

# Traitement et sauvegarde
with open(output_filename, 'w', encoding='utf-8') as f:
    for file_name, parser in pdf_files:
        print(f"Traitement de {file_name}...")
        documents = parser.load_data(file_name)
        f.write(f"# Contenu extrait de : {file_name}\n\n")
        for doc in documents:
            f.write(doc.text + "\n\n")

print(f"✅ Tous les textes ont été extraits et sauvegardés dans : {output_filename}")


Traitement de Wa7ch.pdf...


Started parsing the file under job_id 3bef5486-f45e-495f-acac-2601b268a6fd
.✅ Tous les textes ont été extraits et sauvegardés dans : Wa7ch.md


### Explication :
nest_asyncio est utilisé car LlamaParse fonctionne de manière asynchrone, ce qui peut poser problème dans un notebook Jupyter sans ce correctif.
Le code envoie le PDF au cloud, récupère le texte structuré et le sauvegarde dans un fichier local Wa7ch.md.
### Interprétation :
 On transforme une donnée "binaire" et illisible pour un LLM (le PDF) en un format texte structuré (Markdown) qui préserve les titres et paragraphes. C'est l'étape de Digitalisation/Extraction.

Étape 4 : Création de la Base Vectorielle (Ingestion)

In [ ]:
# 1. Lecture du fichier Markdown
with open("Wa7ch.md", encoding='utf-8') as f:
    markdown_content = f.read()

# 2. Découpage par paragraphes (2 sauts de ligne ou plus)
paragraphs = [p.strip() for p in markdown_content.split('\n\n') if p.strip()]

# 3. Création des documents
documents = [Document(page_content=paragraph) for paragraph in paragraphs]

# 4. Initialisation des embeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

# 5. Configuration de la base vectorielle
persist_directory = "philo_db"
vecdb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name="rag-chroma"
)

# 6. Persistance des données
vecdb.persist()
print("Opération terminée avec succès:")
print(f"- {len(documents)} paragraphes traités")
print(f"- Base vectorielle sauvegardée dans: {persist_directory}")

C:\Users\hiba\AppData\Local\Temp\ipykernel_21892\821983471.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")


### Explication :
On relit le fichier Markdown créé juste avant.
On découpe le texte par paragraphes (double saut de ligne \n\n). Chaque paragraphe devient un "Document".
On charge le modèle d'embedding mxbai-embed-large via Ollama (qui doit tourner localement sur votre machine).
On crée la base de données philo_db qui contient les vecteurs de ces paragraphes.
### Interprétation : 
C'est l'étape d'Indexation. Le texte est transformé en coordonnées mathématiques. Plus tard, quand vous poserez une question, le système comparera les coordonnées de votre question avec celles de ces paragraphes pour trouver la réponse (Recherche Sémantique).

Le flux de travail est organisé de manière linéaire et logique pour un pipeline ETL (Extract, Transform, Load) :
### Configuration (Setup) :
 Clés API et initialisation des objets.
### Extraction (Extract) : 
Envoi du PDF brut -> Réception du Markdown propre (grâce à LlamaParse).
### Transformation (Transform) :
 Découpage du Markdown en petits morceaux (chunks/paragraphes).
### Chargement & Indexation (Load) :
 Calcul des vecteurs (Embeddings) et sauvegarde dans la base de données (ChromaDB).